In [ ]:
# Luke Munday
# 199116045
# 3CB107 - AI Report

In [ ]:
import time
import os
import glob
import copy
import pathlib

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets, models, transforms

In [ ]:
# Define run parmameters

# Choose 'squeezenet', 'vgg' or 'resnet'

model_name = "squeezenet"

transform_image_size = 224

batch_size = 8

num_epochs = 15

mode = 0o777

In [ ]:
# Data dict

# Choose: 'palm' or 'dorsal'

hand_type = 'palm'

data_directory = "./dataset_final/" + hand_type

In [ ]:
# Check device

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# Increase number of unique images by factor of 2 by horizontal flipping,
# but cant vertically flip

# Change colour channels from 0-255 to 0-1, numpy to tensors

train_transformation = transforms.Compose([
    transforms.RandomResizedCrop(transform_image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406], # col = rgb channel
        [0.229, 0.224, 0.225]) # row = mean deviation
])

valid_transformation = transforms.Compose([
        transforms.Resize(transform_image_size),
        transforms.CenterCrop(transform_image_size),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225])
])

In [ ]:
# Datasets

train_dataset = datasets.ImageFolder(os.path.join(data_directory, 'train'), train_transformation)
valid_dataset = datasets.ImageFolder(os.path.join(data_directory, 'valid'), valid_transformation)

# Dataloader

train_dataloader = DataLoader(
    train_dataset,
    batch_size = batch_size,
    shuffle = True, # Shuffle is True so the model is not biased to some categories
    num_workers=4 # How many sub processes to use for loading data into ram
)

valid_dataloader = DataLoader(
    valid_dataset,
    batch_size = batch_size,
    shuffle = True,
    num_workers=4
)

In [ ]:
train_root = pathlib.Path(data_directory + '/train')
data_classes = sorted([item.name.split('/')[-1] for item in train_root.iterdir()])
num_classes = len(data_classes)

In [ ]:
print(data_classes)

In [ ]:
if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=True)
        for parameter in model_ft.parameters():
            parameter.requires_grad = False
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg16_bn(pretrained=True)
        for parameter in model_ft.parameters():
            parameter.requires_grad = False
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=True)
        for parameter in model_ft.parameters():
            parameter.requires_grad = False
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

In [ ]:
model = model_ft.to(device)
print(model)

In [ ]:
params_to_update = model.parameters()
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)
        
optimiser = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
loss_func = nn.CrossEntropyLoss()

In [ ]:
# Calc amount of training and validation images for each class

train_male = len(glob.glob(data_directory + '/train/male/*.jpg'))
train_female = len(glob.glob(data_directory + '/train/female/*.jpg'))
train_count = train_male + train_female

valid_male = len(glob.glob(data_directory + '/valid/male/*.jpg'))
valid_female = len(glob.glob(data_directory + '/valid/female/*.jpg'))
valid_count = valid_male + valid_female

print('train male:', train_male, '\ttrain female:', train_female, '\ttrain count:', train_count)
print('valid male:', valid_male, '\tvalid female:', valid_female, '\tvalid count:', valid_count)

In [ ]:
def run_phase(phase_type, phase_count, phase_dataloader):
    since = time.time()
    phase_accuracy = 0.0
    phase_loss = 0.0
    # image_count = 0

    for images, labels in phase_dataloader:
        inputs = images.to(device)
        labels = labels.to(device)

        optimiser.zero_grad() # Zero out gradients when starting the training loop,
                              # else gradient will point away from objective direction

        with torch.set_grad_enabled(phase_type == 'train'):
            # Forward
            outputs = model(inputs)
            loss = loss_func(outputs, labels)

            _, prediction = torch.max(outputs, 1)

            if phase_type == 'train':
                # Backward
                loss.backward()
                optimiser.step()

        # Calc stats
        phase_accuracy = phase_accuracy + (torch.sum(prediction == labels.data))
        phase_loss = phase_loss + (loss.item() * images.size(0))

        # Update image counter and elpased time
        # time_elapsed = time.time() - since
        # time_min = time_elapsed // 60
        # time_sec = time_elapsed % 60
        # print('\r' + 'Image: ' + str(image_count), '\tElpased Time:', '{:.0f}m {:.0f}s'.format(time_min, time_sec), end='')
        # image_count+=1

    epoch_accuracy = phase_accuracy.double() / phase_count
    epoch_loss = phase_loss / phase_count

    time_elapsed = time.time() - since
    time_min = time_elapsed // 60
    time_sec = time_elapsed % 60
    
    print('\n{} \tLoss: {:.4f} \tAcc: {:.4f} \tElpased Time: {:.0f}m {:.0f}s'.format(phase_type, epoch_loss, epoch_accuracy, time_min, time_sec ))
    
    return epoch_accuracy

In [ ]:
# Model training
def train(model, train_dataloader, valid_dataloader, optimiser, num_epochs):
    best_acc = 0.0
    epoch_accuracies = []
    best_model = copy.deepcopy(model.state_dict())
    since = time.time()

    for epoch in range(num_epochs):

        print('Epoch', str(epoch + 1) + '/' + str(num_epochs))

        # Train model phase
        model.train()
        run_phase('train', train_count, train_dataloader)

        # Eval model phase
        model.eval()
        epoch_acc = run_phase('valid', valid_count, valid_dataloader)
        
        # Check performace
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model = copy.deepcopy(model.state_dict())
            
        # Record performance for analysis
        epoch_accuracies.append(epoch_acc)

        print('\n')

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best Accuracy: {:.4f}' .format(best_acc))
    
    return best_model, epoch_accuracies

In [ ]:
model, epoch_accuracies = train(model, train_dataloader, valid_dataloader, optimiser, num_epochs)

In [ ]:
def create_dir(folder_path):
    try: 
        os.makedirs(os.path.join(folder_path), mode)
    except OSError as error:
        print(error)

In [ ]:
models_dir = 'trained_models/'
create_dir(models_dir)
models_dir = os.path.join(models_dir, model_name)
torch.save(model, models_dir + ('_' + hand_type + '_model.pt'))

In [ ]:
history = [h.cpu().numpy() for h in epoch_accuracies]

In [ ]:
plt.title("Validation Accuracy vs. Number of Training Epochs")
plt.xlabel("Training Epochs")
plt.ylabel("Validation Accuracy")

plt.plot(range(1,num_epochs+1),history,label="Pretrained")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.show()